In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
# from tensorflow.keras.optimizers import SGD
# import tensorflow.keras.legacy.optimizers.SGD
import tensorflow as tf
import random


In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json',encoding="utf-8").read()
intents = json.loads(data_file)


In [3]:
import nltk
nltk.download('punkt')


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

700 documents
52 classes ['\tPlant Protection\t', 'Fertilizer Use and Availability', 'General_About_You', 'General_Agent_Capabilities', 'General_Connect_to_Agent', 'General_Ending', 'General_Greetings', 'General_Human_or_Bot', 'General_Negative_Feedback', 'General_Positive_Feedback', 'Income-based-policies', 'Kisan-Credit-Card', 'Loan-policies', 'Medicinal and Aromatic Plants', 'Nutrient Management', 'PM-Kisan-Maandhan-Scheme', 'Pesticides-disease-downy-mildew', 'Rainfed-Area-Development-Programme', 'Seeds and Planting Material', 'Symptoms-disease-downy-mildew', 'Varieties', 'activities-in-pradhan-mantri-krishi-sinchayee-yojana', 'brands-for-bayleton', 'brands-for-bordeaux-mixture', 'brands-for-carbendazim', 'brands-for-copper-oxychloride', 'brands-for-fosetyl', 'brands-for-mancozeb', 'brands-for-metalaxyl', 'brands-for-streptocycline', 'brands-for-wettable sulphur', 'choice-brands', 'choice-diseases', 'choice-gov-policies', 'choice-pest', 'disease-anthracnose', 'disease-bacterial-leaf

In [6]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


C:\Users\HP\AppData\Local\Temp\ipykernel_24868\3106589240.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [7]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
sgd = tf.keras.optimizers.legacy.SGD(learning_rate=0.1)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=80, batch_size=15, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/80
47/47 [==============================] - 6s 14ms/step - loss: 3.8452 - accuracy: 0.0871
Epoch 2/80
47/47 [==============================] - 1s 13ms/step - loss: 3.5527 - accuracy: 0.1429
Epoch 3/80
47/47 [==============================] - 1s 11ms/step - loss: 3.3448 - accuracy: 0.1457
Epoch 4/80
47/47 [==============================] - 1s 12ms/step - loss: 3.2303 - accuracy: 0.1629
Epoch 5/80
47/47 [==============================] - 1s 15ms/step - loss: 3.1460 - accuracy: 0.1829
Epoch 6/80
47/47 [==============================] - 1s 14ms/step - loss: 3.0189 - accuracy: 0.2200
Epoch 7/80
47/47 [==============================] - 1s 20ms/step - loss: 2.8863 - accuracy: 0.2157
Epoch 8/80
47/47 [==============================] - 1s 17ms/step - loss: 2.7670 - accuracy: 0.2714
Epoch 9/80
47/47 [==============================] - 1s 22ms/step - loss: 2.6953 - accuracy: 0.2743
Epoch 10/80
47/47 [==============================] - 1s 16ms/step - loss: 2.6054 - accuracy: 0.2814
Epoch 11/

In [8]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json',encoding = "utf-8").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [9]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [10]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [11]:
from translate import Translator

In [12]:
class difflanguage:
    def lang(self, Language):

        default = "English"

        return getattr(self, 'case_' + str(Language), lambda: default)()

    def case_1(self):
        translator= Translator(from_lang="english",to_lang=("gujarati"))
        translation = translator.translate("Good Morning")
        return translation

 

    def case_2(self):
        translator= Translator(from_lang="english",to_lang=("marathi"))
        translation = translator.translate("Good Morning")
        return translation



    def case_3(self):
        translator= Translator(from_lang="english",to_lang=("german"))
        translation = translator.translate("Good Morning")
        return translation

 

    def case_4(self):
        translator= Translator(from_lang="english",to_lang=("french"))
        translation = translator.translate("Good Morning")
        return translation



    def case_5(self):
        translator= Translator(from_lang="english",to_lang=("tamil"))
        translation = translator.translate("Good Morning")
        return translation
    
    
   
my_switch = difflanguage()
my_switch.lang(1)

'ગુડ મોર્નીંગ'

In [13]:
import tkinter as tk
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="Black", font=("Verdana", 14))
        res = chatbot_response(msg)
        my_switch = difflanguage()
        ChatLog.insert(END, "EDITH: " + res + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = tk.Tk()
base.title("Multilingual Chatbot")
base.geometry("700x800")
base.resizable(width=FALSE, height=FALSE)
ChatLog = Text(base, bd=0, bg="#8E8D8D", height="10", width="60", font="Arial")
ChatLog.config(state=DISABLED)
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
SendButton = Button(base, font=("Verdana",18,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#5CA1CB", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )
EntryBox = Text(base, bd=0, bg="WHITE",width="29", height="5", font=("Verdana",18))
scrollbar.place(x=682,y=6, height=800)
ChatLog.place(x=6,y=6, height=673, width=800)
EntryBox.place(x=301, y=679, height=120, width=380)
SendButton.place(x=6, y=679, height=120,width=300)
base.mainloop()

1/1 [==============================] - 0s 78ms/step
